##setting up

In [0]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

import pandas as pd
import matplotlib.pyplot as plt
import io
import unicodedata
import numpy as np
import re
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


##Load the corpus file form Google Drive into Colab


In [0]:
# Upload from google drive
from google.colab import files
uploaded = files.upload()
print("len(uploaded.keys():", len(uploaded.keys()))

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

# Get dataframe of the tweets in the labeled dataset 
df_tweets = pd.read_csv('labeled_data.csv',encoding = 'utf-8',sep=',',index_col=0)

#selecting the right column for data preprocessing 
tweets_pd_raw = df_tweets.iloc[:,5]

#inspecting the selected column
tweets_pd_raw.head(5)

Saving labeled_data.csv to labeled_data.csv
len(uploaded.keys(): 1
User uploaded file "labeled_data.csv" with length 2546446 bytes


0    !!! RT @mayasolovely: As a woman you shouldn't...
1    !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2    !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3    !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4    !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
Name: tweet, dtype: object

##Replace, tokenize sentences and words, remove stopwords, use stemmer & lemmatizer



In [0]:
#setting up the libaries
from nltk.corpus import stopwords
from nltk.stem.porter import *
stop = stopwords.words('english')

#replacing urls, excessive whitespaces, and mentions
def replacing(text):
  space_pattern = '\s+'
  giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
  mention_regex = '@[\w\-]+'
  replaced_text = re.sub(space_pattern, ' ', text)
  replaced_text = re.sub(giant_url_regex, '', replaced_text)
  replaced_text = re.sub(mention_regex, '', replaced_text)
  return replaced_text

tweets_pd_replaced = tweets_pd_raw.apply(replacing)

#tokenize the text and remove punctuations, set to lowercase and
#return a list of tokenized text
def tokenization(text):
  tokenized_text = " ".join(re.split("[^a-zA-Z]*", text.lower())).strip()
  return tokenized_text

tweets_tokenized = tweets_pd_replaced.apply(tokenization)

#stem the tokenized text
stemmer = PorterStemmer()
def stem(text):
  stemmed = [stemmer.stem(t) for t in text.split()]
  return stemmed

tweets_stemmed = tweets_tokenized.apply(stem)

#paste the processed text back to the labeled dataset
df_tweets.iloc[:,5] = tweets_stemmed.values

df_tweets.head(5)


/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,"[rt, as, a, woman, you, shouldn, t, complain, ..."
1,3,0,3,0,1,"[rt, boy, dat, cold, tyga, dwn, bad, for, cuff..."
2,3,0,3,0,1,"[rt, dawg, rt, you, ever, fuck, a, bitch, and,..."
3,3,0,2,1,1,"[rt, she, look, like, a, tranni]"
4,6,0,6,0,1,"[rt, the, shit, you, hear, about, me, might, b..."


##Using the Tfidf Matrix to generate the Features of our Corpus

In [0]:
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from keras.models import Model
from sklearn.metrics import classification_report
from keras import layers, models, optimizers
import sklearn.metrics as metrics

#tweet column is a column of lists, and not text. 
#Tfidf Vectorizer works on text. So  to get the score, I have to put tokens back to strings
df_tweets['tweet']=[" ".join(review) for review in df_tweets['tweet'].values]

x_train, x_test, y_train, y_test = train_test_split(df_tweets['tweet'], df_tweets['class'], test_size=0.3, random_state=1)

tfidf = TfidfVectorizer()
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)


In [0]:
def rnnModel(Xtrain, Ytrain, Xtest):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(1000, 64),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(10)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.fit(Xtrain, Ytrain, batch_size=30, epochs=10)
    
    test_loss, test_acc = model.evaluate(Xtest, y_test)
    print("\nAccuracy : ",test_acc)
    
    predictions = model.predict(Xtest).ravel()
    
    target_names = ['class 0', 'class 1']
    print(classification_report(y_test, predictions.round(), target_names=target_names))

    print("\nRNN with  Vectors + TF-IDF: ")

rnnModel(x_train_tfidf, y_train, x_test_tfidf)


Epoch 1/10


InvalidArgumentError: ignored